In [1]:
print("="*70)
print("🔍 MULTI-DOMAIN SEARCH ENGINE - SETUP")
print("="*70)
print("\n📦 Installing required packages...\n")

import subprocess
import sys

def install_package(package):
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✅ {package} installed")

packages = ['pandas', 'nltk']
for pkg in packages:
    install_package(pkg)

print("\n📚 Downloading NLTK data...")
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("✅ NLTK data ready!")

print("\n" + "="*70)
print("✅ SETUP COMPLETE!")
print("="*70)

🔍 MULTI-DOMAIN SEARCH ENGINE - SETUP

📦 Installing required packages...

✅ pandas already installed
✅ nltk already installed

📚 Downloading NLTK data...
✅ NLTK data ready!

✅ SETUP COMPLETE!


In [2]:
import pandas as pd
import json
import re
import time
from collections import defaultdict, Counter
from typing import Dict, List, Tuple
import hashlib
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [3]:
print("\n" + "="*70)
print("⚙️ DOMAIN CONFIGURATION")
print("="*70)

# Configure your datasets here
DOMAIN_CONFIG = {
    'news': {
        'name': 'NEWS',
        'description': 'News articles from various categories',
        'file': r"C:\Users\kanish pc\Downloads\domain specific search engine\data\News_Category_Dataset_v3.csv",
        'file_type': 'csv',  
        'limit': 5000,
        'columns': {
            'title': 'title',              
            'content': 'description',      
            'category': 'news',      
        }
    },
     'sports': {
        'name': 'SPORTS',
        'description': 'Sports news and updates',
        'file': r"C:\Users\kanish pc\Downloads\domain specific search engine\data\bbc_sports.csv",
        'file_type': 'csv',
        'limit': 5000,
        'columns': {
            'title': 'title',              
            'content': 'description',      
            'category': 'sports',          
        }
    },
     'business': {
        'name': 'BUSINESS',
        'description': 'Business and financial news', 
        'file': r"C:\Users\kanish pc\Downloads\domain specific search engine\data\Combined_News_DJIA.csv",
        'file_type': 'csv',
        'limit': 5000,
        'columns': {
            'title': 'title',              
            'content': 'description',      
            'category': 'business',        
        }
    }
}

print("\n✅ Domain configuration loaded:")
for domain_key, config in DOMAIN_CONFIG.items():
    print(f"\n📁 Domain: {config['name'].upper()}")
    print(f"   File: {config['file']}")
    print(f"   Type: {config['file_type']}")
    print(f"   Limit: {config['limit']} articles")

print("\n" + "="*70)



⚙️ DOMAIN CONFIGURATION

✅ Domain configuration loaded:

📁 Domain: NEWS
   File: C:\Users\kanish pc\Downloads\domain specific search engine\data\News_Category_Dataset_v3.csv
   Type: csv
   Limit: 5000 articles

📁 Domain: SPORTS
   File: C:\Users\kanish pc\Downloads\domain specific search engine\data\bbc_sports.csv
   Type: csv
   Limit: 5000 articles

📁 Domain: BUSINESS
   File: C:\Users\kanish pc\Downloads\domain specific search engine\data\Combined_News_DJIA.csv
   Type: csv
   Limit: 5000 articles



In [4]:
class InvertedIndex:
    """Inverted Index with TF-IDF ranking"""
    
    def __init__(self):
        self.index = defaultdict(list)
        self.documents = {}
        self.doc_count = 0
        self.stop_words = set(stopwords.words('english'))
        self.total_words = 0
        self.unique_words = set()
        
    def add_document(self, doc_id: str, content: str, metadata: Dict):
        """Add document to index"""
        self.documents[doc_id] = metadata
        tokens = self._tokenize(content)
        
        # Track unique words
        self.unique_words.update(tokens)
        self.total_words += len(tokens)
        
        # Build term frequency map
        term_positions = defaultdict(list)
        for pos, token in enumerate(tokens):
            term_positions[token].append(pos)
        
        # Update inverted index
        for term, positions in term_positions.items():
            self.index[term].append({
                'doc_id': doc_id,
                'term_freq': len(positions),
                'positions': positions
            })
        
        self.doc_count += 1
    
    def search(self, query: str, limit: int = 10) -> List[Dict]:
        """Search using TF-IDF ranking"""
        query_tokens = self._tokenize(query)
        
        if not query_tokens:
            return []
        
        doc_scores = defaultdict(float)
        
        for token in query_tokens:
            if token in self.index:
                idf = self._calculate_idf(token)
                
                for posting in self.index[token]:
                    doc_id = posting['doc_id']
                    tf = posting['term_freq']
                    doc_scores[doc_id] += tf * idf
        
        sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        
        results = []
        for doc_id, score in sorted_docs[:limit]:
            result = self.documents[doc_id].copy()
            result['score'] = score
            results.append(result)
        
        return results
    
    def _tokenize(self, text: str) -> List[str]:
        """Tokenize and clean text"""
        tokens = re.findall(r'\b\w+\b', text.lower())
        return [t for t in tokens if len(t) > 2 and t not in self.stop_words]
    
    def _calculate_idf(self, term: str) -> float:
        """Calculate inverse document frequency"""
        import math
        df = len(self.index[term])
        return math.log((self.doc_count + 1) / (df + 1))
    
    def get_stats(self) -> Dict:
        """Get indexing statistics"""
        categories = Counter(
            doc.get('category', 'Unknown') 
            for doc in self.documents.values()
        )
        
        return {
            'total_documents': self.doc_count,
            'total_words_processed': self.total_words,
            'unique_words': len(self.unique_words),
            'total_terms_indexed': len(self.index),
            'categories': dict(categories),
            'avg_doc_length': self.total_words / max(self.doc_count, 1)
        }

print("✅ InvertedIndex class created!")

✅ InvertedIndex class created!


In [5]:
class DatasetLoader:
    """Load and prepare datasets for indexing"""
    
    @staticmethod
    def load_dataset(config: Dict) -> List[Dict]:
        """Load dataset based on configuration"""
        file_path = config['file']
        file_type = config['file_type']
        limit = config['limit']
        columns = config['columns']
        
        print(f"\n📁 Loading dataset: {file_path}")
        print(f"   Type: {file_type}")
        print(f"   Target: {limit} articles")
        
        try:
            if file_type == 'json':
                # Load JSON file
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = []
                    for i, line in enumerate(f):
                        if i >= limit:
                            break
                        try:
                            data.append(json.loads(line))
                        except:
                            continue
                df = pd.DataFrame(data)
                
            elif file_type == 'csv':
                # Load CSV file
                df = pd.read_csv(file_path, nrows=limit, encoding='utf-8')
            
            print(f"✅ Loaded {len(df)} records from file")
            
            # Convert to standard format
            documents = []
            for idx, row in df.iterrows():
                # Get values with fallbacks
                title = str(row.get(columns['title'], f"Article {idx+1}"))
                content = str(row.get(columns['content'], ""))
                category = str(row.get(columns['category'], "General"))
                
                # Skip if no content
                if not content or content == 'nan':
                    continue
                
                doc = {
                    'id': idx + 1,
                    'title': title,
                    'content': content,
                    'category': category,
                    'url': f"article-{idx+1}",
                    'domain': config['name']
                }
                documents.append(doc)
            
            print(f"✅ Converted {len(documents)} valid documents")
            return documents
            
        except FileNotFoundError:
            print(f"❌ ERROR: File '{file_path}' not found!")
            print(f"💡 Please download the dataset and place it in this folder")
            return []
        except Exception as e:
            print(f"❌ ERROR loading dataset: {e}")
            return []

print("✅ DatasetLoader class created!")

✅ DatasetLoader class created!


In [6]:
class DomainSearchEngine:
    """Search engine for a specific domain"""
    
    def __init__(self, domain_name: str, config: Dict):
        self.domain_name = domain_name
        self.config = config
        self.index = InvertedIndex()
        self.documents = []
        self.crawl_stats = {}
        
    def crawl_and_index(self):
        """Crawl dataset and build index"""
        print("\n" + "="*70)
        print(f"🚀 STARTING CRAWL & INDEX FOR DOMAIN: {self.domain_name.upper()}")
        print("="*70)
        
        # Load dataset
        print(f"\n📊 Phase 1: Loading Dataset")
        print("-" * 70)
        start_time = time.time()
        self.documents = DatasetLoader.load_dataset(self.config)
        load_time = time.time() - start_time
        
        if not self.documents:
            print("\n❌ No documents loaded! Check file path and format.")
            return False
        
        print(f"⏱️  Loading completed in {load_time:.2f} seconds")
        
        # Build index
        print(f"\n📊 Phase 2: Building Inverted Index")
        print("-" * 70)
        start_time = time.time()
        
        for i, doc in enumerate(self.documents, 1):
            doc_id = str(doc['id'])
            searchable_text = f"{doc['title']} {doc['content']}"
            self.index.add_document(doc_id, searchable_text, doc)
            
            # Progress indicator
            if i % 500 == 0:
                print(f"   Indexed: {i}/{len(self.documents)} documents...")
        
        index_time = time.time() - start_time
        print(f"⏱️  Indexing completed in {index_time:.2f} seconds")
        
        # Get statistics
        stats = self.index.get_stats()
        self.crawl_stats = {
            'load_time': load_time,
            'index_time': index_time,
            'total_time': load_time + index_time,
            **stats
        }
        
        # Display statistics
        self._display_stats()
        
        return True
    
    def _display_stats(self):
        """Display crawl and index statistics"""
        print("\n" + "="*70)
        print(f"📊 CRAWL & INDEX STATISTICS - {self.domain_name.upper()} DOMAIN")
        print("="*70)
        
        stats = self.crawl_stats
        
        print(f"\n⏱️  TIMING:")
        print(f"   • Dataset Loading Time: {stats['load_time']:.2f} seconds")
        print(f"   • Index Building Time: {stats['index_time']:.2f} seconds")
        print(f"   • Total Processing Time: {stats['total_time']:.2f} seconds")
        
        print(f"\n📄 DOCUMENTS:")
        print(f"   • Total Documents Indexed: {stats['total_documents']:,}")
        print(f"   • Average Document Length: {stats['avg_doc_length']:.0f} words")
        
        print(f"\n📝 WORD STATISTICS:")
        print(f"   • Total Words Processed: {stats['total_words_processed']:,}")
        print(f"   • Unique Words Found: {stats['unique_words']:,}")
        print(f"   • Terms in Index: {stats['total_terms_indexed']:,}")
        print(f"   • Vocabulary Diversity: {(stats['unique_words']/stats['total_words_processed']*100):.2f}%")
        
        print(f"\n📁 CATEGORIES ({len(stats['categories'])}):")
        for category, count in sorted(stats['categories'].items(), 
                                     key=lambda x: x[1], reverse=True)[:10]:
            print(f"   • {category}: {count} documents")
        
        print("\n" + "="*70)
        print("✅ DOMAIN READY FOR SEARCHING!")
        print("="*70)
    
    def search(self, query: str, limit: int = 10) -> List[Dict]:
        """Search within this domain"""
        return self.index.search(query, limit)
    
    def get_stats(self) -> Dict:
        """Get domain statistics"""
        return self.crawl_stats

print("✅ DomainSearchEngine class created!")

✅ DomainSearchEngine class created!


In [7]:
class MultiDomainSearchSystem:
    """Manages multiple domain search engines"""
    
    def __init__(self, domain_configs: Dict):
        self.domain_configs = domain_configs
        self.domains = {}
        self.current_domain = None
        
    def initialize_domain(self, domain_key: str) -> bool:
        """Initialize and index a specific domain"""
        if domain_key not in self.domain_configs:
            print(f"❌ Unknown domain: {domain_key}")
            print(f"Available domains: {', '.join(self.domain_configs.keys())}")
            return False
        
        config = self.domain_configs[domain_key]
        
        print(f"\n🎯 Initializing Domain: {config['name']}")
        print(f"📝 Description: {config['description']}")
        
        # Create domain search engine
        engine = DomainSearchEngine(config['name'], config)
        
        # Crawl and index
        success = engine.crawl_and_index()
        
        if success:
            self.domains[domain_key] = engine
            self.current_domain = domain_key
            return True
        
        return False
    
    def select_domain(self, domain_key: str) -> bool:
        """Select a domain for searching"""
        if domain_key not in self.domains:
            print(f"\n⚠️  Domain '{domain_key}' not initialized yet.")
            print(f"Initializing now...")
            return self.initialize_domain(domain_key)
        
        self.current_domain = domain_key
        print(f"\n✅ Selected domain: {self.domain_configs[domain_key]['name']}")
        return True
    
    def search(self, query: str, limit: int = 10) -> List[Dict]:
        """Search in current domain"""
        if not self.current_domain:
            print("❌ No domain selected! Please select a domain first.")
            return []
        
        engine = self.domains[self.current_domain]
        return engine.search(query, limit)
    
    def get_available_domains(self) -> List[str]:
        """Get list of available domains"""
        return list(self.domain_configs.keys())
    
    def get_initialized_domains(self) -> List[str]:
        """Get list of initialized domains"""
        return list(self.domains.keys())

print("✅ MultiDomainSearchSystem class created!")

✅ MultiDomainSearchSystem class created!


In [8]:
print("\n" + "="*70)
print("🚀 INITIALIZING MULTI-DOMAIN SEARCH SYSTEM")
print("="*70)

# Create the multi-domain system
search_system = MultiDomainSearchSystem(DOMAIN_CONFIG)

print("\n✅ System initialized!")
print(f"\n📁 Available Domains:")
for domain in search_system.get_available_domains():
    config = DOMAIN_CONFIG[domain]
    print(f"   • {domain}: {config['name']} - {config['description']}")

print("\n" + "="*70)
print("💡 Ready to load domains!")
print("="*70)


🚀 INITIALIZING MULTI-DOMAIN SEARCH SYSTEM

✅ System initialized!

📁 Available Domains:
   • news: NEWS - News articles from various categories
   • sports: SPORTS - Sports news and updates
   • business: BUSINESS - Business and financial news

💡 Ready to load domains!


In [9]:
def search_interface():
    """Interactive search interface"""
    
    print("\n" + "="*70)
    print("🔍 MULTI-DOMAIN SEARCH ENGINE")
    print("="*70)
    
    while True:
        print("\n📁 SELECT DOMAIN:")
        print("-" * 70)
        domains = search_system.get_available_domains()
        for i, domain in enumerate(domains, 1):
            config = DOMAIN_CONFIG[domain]
            status = "✅ LOADED" if domain in search_system.get_initialized_domains() else "⏳ NOT LOADED"
            print(f"   {i}. {domain.upper()} - {config['description']} [{status}]")
        print(f"   {len(domains)+1}. EXIT")
        
        print("\n" + "-" * 70)
        choice = input("Enter domain number or name (news/sports/business): ").strip().lower()
        
        # Exit option
        if choice == str(len(domains)+1) or choice == 'exit' or choice == 'quit':
            print("\n👋 Thanks for using Multi-Domain Search Engine!")
            break
        
        # Determine domain
        selected_domain = None
        if choice.isdigit() and 1 <= int(choice) <= len(domains):
            selected_domain = domains[int(choice)-1]
        elif choice in domains:
            selected_domain = choice
        else:
            print("❌ Invalid selection!")
            continue
        
        # Select/Initialize domain
        if not search_system.select_domain(selected_domain):
            continue
        
        # Search loop for selected domain
        config = DOMAIN_CONFIG[selected_domain]
        while True:
            print("\n" + "="*70)
            print(f"🔍 SEARCH IN: {config['name']} DOMAIN")
            print("="*70)
            print("💡 Commands: Type query to search | 'stats' for statistics | 'back' to change domain")
            
            query = input("\n🔎 Search: ").strip()
            
            if not query:
                continue
            
            if query.lower() == 'back':
                break
            
            if query.lower() == 'stats':
                engine = search_system.domains[selected_domain]
                engine._display_stats()
                continue
            
            # Perform search
            print(f"\n⏳ Searching in {config['name']} domain...")
            results = search_system.search(query, limit=10000)
            
            # Display results
            print("\n" + "="*70)
            print(f"📊 SEARCH RESULTS - Query: '{query}'")
            print("="*70)
            
            if not results:
                print("\n❌ No results found.")
                print("💡 Try different keywords or check if domain is properly loaded.")
            else:
                print(f"\n✅ Found {len(results)} results:\n")
                for i, result in enumerate(results, 1):
                    print(f"{i}. {result.get('title', 'Untitled')}")
                    print(f"   📊 Relevance Score: {result.get('score', 0):.2f}")
                    print(f"   🏷️  Category: {result.get('category', 'N/A')}")
                    print(f"   📝 Content Preview: {result.get('content', '')[:150]}...")
                    print()
            
            print("="*70)

print("✅ Search interface function ready!")
print("\n💡 TO START SEARCHING:")
print("   Run: search_interface()")

✅ Search interface function ready!

💡 TO START SEARCHING:
   Run: search_interface()


In [10]:
search_interface()


🔍 MULTI-DOMAIN SEARCH ENGINE

📁 SELECT DOMAIN:
----------------------------------------------------------------------
   1. NEWS - News articles from various categories [⏳ NOT LOADED]
   2. SPORTS - Sports news and updates [⏳ NOT LOADED]
   3. BUSINESS - Business and financial news [⏳ NOT LOADED]
   4. EXIT

----------------------------------------------------------------------


Enter domain number or name (news/sports/business):  news



⚠️  Domain 'news' not initialized yet.
Initializing now...

🎯 Initializing Domain: NEWS
📝 Description: News articles from various categories

🚀 STARTING CRAWL & INDEX FOR DOMAIN: NEWS

📊 Phase 1: Loading Dataset
----------------------------------------------------------------------

📁 Loading dataset: C:\Users\kanish pc\Downloads\domain specific search engine\data\News_Category_Dataset_v3.csv
   Type: csv
   Target: 5000 articles
✅ Loaded 5000 records from file
✅ Converted 5000 valid documents
⏱️  Loading completed in 0.52 seconds

📊 Phase 2: Building Inverted Index
----------------------------------------------------------------------
   Indexed: 500/5000 documents...
   Indexed: 1000/5000 documents...
   Indexed: 1500/5000 documents...
   Indexed: 2000/5000 documents...
   Indexed: 2500/5000 documents...
   Indexed: 3000/5000 documents...
   Indexed: 3500/5000 documents...
   Indexed: 4000/5000 documents...
   Indexed: 4500/5000 documents...
   Indexed: 5000/5000 documents...
⏱️  In


🔎 Search:  bomb and wars



⏳ Searching in NEWS domain...

📊 SEARCH RESULTS - Query: 'bomb and wars'

✅ Found 18 results:

1. WATCH: Disposal experts defuse unexploded bomb in Ukraine
   📊 Relevance Score: 11.76
   🏷️  Category: General
   📝 Content Preview: An unexploded 500kg (1102lb) bomb was discovered in Chernihiv in northern Ukraine....

2. What is a thermobaric or vacuum bomb?
   📊 Relevance Score: 11.76
   🏷️  Category: General
   📝 Content Preview: What is a thermobaric or vacuum bomb, and why is Russia using them in the war in Ukraine?...

3. Ukraine war: 60 people killed after bomb hits school, Zelensky says
   📊 Relevance Score: 11.76
   🏷️  Category: General
   📝 Content Preview: About 90 people had been sheltering in the building which Ukraine says was hit by a Russian bomb....

4. Manchester Arena bomb family 'living the best lives they can'
   📊 Relevance Score: 11.76
   🏷️  Category: General
   📝 Content Preview: Eve Senior's photograph after the Manchester Arena bomb five years ago was seen aro

...


⏳ Searching in BUSINESS domain...

📊 SEARCH RESULTS - Query: 'oil and gas'

✅ Found 114 results:

1. War in Ukraine: Russia says it may cut gas supplies if oil ban goes ahead
   📊 Relevance Score: 14.58
   🏷️  Category: General
   📝 Content Preview: A top official says Russia may close its gas lines to Germany if the West halts oil imports....

2. War in Ukraine: Russia says it may cut gas supplies if oil ban goes ahead
   📊 Relevance Score: 14.58
   🏷️  Category: General
   📝 Content Preview: A top official says Russia may close its gas lines to Germany if the West halts oil imports....

3. Why can't the US stop soaring oil and gas prices?
   📊 Relevance Score: 14.58
   🏷️  Category: General
   📝 Content Preview: As prices rise, oil and gas firms in the US want to capitalise - but the industry has changed....

4. What is the windfall tax on oil and gas companies?
   📊 Relevance Score: 14.58
   🏷️  Category: General
   📝 Content Preview: The government has extended the windfall tax on


🔎 Search:  back



📁 SELECT DOMAIN:
----------------------------------------------------------------------
   1. NEWS - News articles from various categories [✅ LOADED]
   2. SPORTS - Sports news and updates [✅ LOADED]
   3. BUSINESS - Business and financial news [✅ LOADED]
   4. EXIT

----------------------------------------------------------------------


Enter domain number or name (news/sports/business):  exit



👋 Thanks for using Multi-Domain Search Engine!
